<a href="https://colab.research.google.com/github/IbtissamSaadi/vit-pytorch/blob/main/Viit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install vit_pytorch
!pip install timm

In [ ]:
import torch, torchvision
import numpy as np
import pandas as pd
#from tqdm import tqdm
import PIL.Image as Image
#import seaborn as sns
#from pylab import rcParams
import matplotlib.pyplot as plt
#from matplotlib import rc
from matplotlib.ticker import MaxNLocator
from torch.optim import lr_scheduler
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix, classification_report
#from glob import glob
#import shutil
import timm
from collections import defaultdict
from torch import nn, optim
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models, datasets
from vit_pytorch import ViT

import time

#%matplotlib inline
#%config InlineBackend.figure_format='retina'
#sns.set(style='whitegrid', palette='muted', font_scale=1.2)
#HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
#sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
#rcParams['figure.figsize'] = 12, 8

torch.manual_seed(42)
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Dpath = '/content/drive/MyDrive/CK+48'
#Bs_Train = 100
#Bs_Test = 1000
"""
tform_mnist = torchvision.transforms.Compose([
    #torchvision.transforms.RandomHorizontalFlip(),
    #torchvision.transforms.RandomRotation(30),
    #torchvision.transforms.CenterCrop((16)),
    #torchvision.transforms.RandomAffine(degrees=40, scale=(.3, 1.1), shear=0.15),
    #torchvision.transforms.Resize((48,48)),                                          
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,),(0.5,))
])
"""
"""
"""
path_train = '/content/drive/MyDrive/FER2013 small dataset/train'
path_vaild = '/content/drive/MyDrive/FER2013 small dataset/test'
#apply multiple transformer after each other
transforms_vaild = torchvision.transforms.Compose([
                                     torchvision.transforms.Resize((224,)),
                                     #torchvision.transforms.Grayscale(),
                                     torchvision.transforms.ToTensor(),
                                     torchvision.transforms.Normalize((0.5026,), (0.2595,))
                                     ])

# For training, we add some augmentation. Networks are too powerful and would overfit.
transforms_train = torchvision.transforms.Compose([
                                      torchvision.transforms.Resize((224,)),
                                      #torchvision.transforms.Grayscale(),             
                                      torchvision.transforms.RandomHorizontalFlip(),
                                      #torchvision.transforms.RandomRotation(30),
                                      #torchvision.transforms.CenterCrop((16)),
                                      #torchvision.transforms.RandomAffine(degrees=40, scale=(.3, 1.1), shear=0.15),
                                      torchvision.transforms.ToTensor(),
                                      torchvision.transforms.Normalize((0.5066,), (0.2627,))
                                     ])
"""
transforms_train = transforms.Compose([
         transforms.Grayscale(),#Use ImageFolder to expand to three channels by default, just change it back
         transforms.RandomHorizontalFlip(),#Random Flip
         transforms.ColorJitter(brightness=0.5, contrast=0.5),#Adjust brightness and contrast randomly
    transforms.ToTensor()
])
transforms_vaild = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
])
"""
data_train = torchvision.datasets.ImageFolder(root=path_train,transform=transforms_train)
data_vaild = torchvision.datasets.ImageFolder(root=path_vaild,transform=transforms_vaild)
print("Classes: ")
classes = data_train.classes 
print(classes)
train_set = torch.utils.data.DataLoader(dataset=data_train,batch_size=128,shuffle=True)
vaild_set = torch.utils.data.DataLoader(dataset=data_vaild,batch_size=128,shuffle=False)
"""
orig_set = datasets.ImageFolder(root=Dpath )  #  dataset
n = len(orig_set)  # total number of examples
print("Classes: ")
print(orig_set.classes)
n_test = int(0.15 * n)  # take ~10% for test
test_set = orig_set(test_transform)
train_set = orig_set(train_transform)

test_set = torch.utils.data.Subset(orig_set, range(n_test))  # take first 30%
print("test ",len(test_set))
train_set = torch.utils.data.Subset(orig_set, range(n_test, n))  # take the rest 
print("train ",len(train_set))

train_loader = torch.utils.data.DataLoader(train_set, batch_size=16, 
                                          shuffle = True )
test_loader = torch.utils.data.DataLoader(test_set, batch_size=16, 
                                          shuffle = False)
"""

"""
DOWNLOAD_PATH = '/data/mnist'
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_TEST = 1000

transform_mnist = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize((0.1307,), (0.3081,))])

train_set = torchvision.datasets.MNIST(DOWNLOAD_PATH, train=True, download=True,
                                       transform=transform_mnist)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)

test_set = torchvision.datasets.MNIST(DOWNLOAD_PATH, train=False, download=True,
                                      transform=transform_mnist)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE_TEST, shuffle=True)
"""
"""
vit_model = timm.create_model('vit_base_patch16_224', pretrained=True)

for param in vit_model.parameters():
      param.requires_grad = False

num_inputs = vit_model.head.in_features
last_layer = nn.Linear(num_inputs, len(orig_set.classes))
vit_model.head = last_layer
"""
"""
def create_model(n_classes):
  model = timm.create_model('vit_base_patch16_224', pretrained=True)
  n_features = model.head.in_features
  model.head = nn.Linear(n_features, n_classes)
  return model
"""


def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  #scheduler,
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0

  for inputs, labels in data_loader:
    #inputs = inputs.to(device)
    #labels = labels.to(device)
    #start forward pass
    outputs = model(inputs)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)
    correct_predictions += torch.sum(preds == labels)
    losses.append(loss.item())
    #backward pass
    loss.backward()#calculate gradient
    optimizer.step()#update wightes
    optimizer.zero_grad()#empty our gradients for the next iteration
  #scheduler.step()
  return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  #to track the history for evaluate the model and get the accuracy
  #we don't calculate the gradient
  with torch.no_grad():
    for inputs, labels in data_loader:
      #inputs = inputs.to(device)
      #labels = labels.to(device)
      outputs = model(inputs)
      #actual predict _ we don't need it, predict
      _, preds = torch.max(outputs, dim=1)#choose highest prob for every output and class them to the right class
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses)

def train_model(model, train_loaders, test_loaders, n_epochs):
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  #scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
  loss_fn =  nn.CrossEntropyLoss()
  history = defaultdict(list)

  dataset_sizes_train = len(data_train)
  dataset_sizes_test = len(data_vaild)
  best_accuracy = 0
  for epoch in range(n_epochs):
    print(f'Epoch {epoch+1 }/{n_epochs}')
    print('-' * 10)
    train_acc, train_loss = train_epoch(
      model,
      train_loaders,
      loss_fn,
      optimizer,
      #scheduler,
      dataset_sizes_train
    )
    print(f'Train loss {train_loss} accuracy {train_acc}')
    val_acc, val_loss = eval_model(
      model,
      test_loaders,
      loss_fn,
      dataset_sizes_test
    )
    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
    
    if val_loss > 2.50:
       break
   
    if val_acc > best_accuracy:
      best_accuracy = val_acc
      #es = 0
      torch.save(model.state_dict(), 'best_model_state.pth')
    #else:
     # es += 1
      #print("Counter {} of 5".format(es))

      #if es > 10:
       # print(f'Early stopping with best_acc: {best_accuracy}')
        #break
  print(f'Best val accuracy: {best_accuracy} , Accuracy: {100 * best_accuracy}')
  model.load_state_dict(torch.load('best_model_state.pth'))
  return model, history
"""
base_model = ViT(image_size=48, patch_size=8, num_classes=7, channels=1,
             dim=256, depth=6, heads=8, mlp_dim=256, dropout = 0.1, emb_dropout = 0.1)
"""

vit_model = timm.create_model('vit_base_patch16_224', pretrained=True)
for param in vit_model.parameters():
      param.requires_grad = False

num_inputs = vit_model.head.in_features
vit_model.head = nn.Linear(num_inputs, 7)

start_time = time.time()
base_model, history = train_model(vit_model, train_set, vaild_set, 30)

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Classes: 
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Epoch 1/30
----------
Train loss 1.6033198616721414 accuracy 0.46035559827006245
Val   loss 1.3646010100841521 accuracy 0.48961937716262977

Epoch 2/30
----------
Train loss 1.211252055384896 accuracy 0.5744834214320038
Val   loss 1.3198442041873932 accuracy 0.48875432525951557

Epoch 3/30
----------
Train loss 1.0866859826174649 accuracy 0.6086016338298895
Val   loss 1.1976202100515365 accuracy 0.5536332179930796

Epoch 4/30
----------
Train loss 1.0261591510339216 accuracy 0.6280634310427679
Val   loss 1.1880554020404817 accuracy 0.5493079584775087

Epoch 5/30
----------
Train loss 0.990105253277403 accuracy 0.6427198462277751
Val   loss 1.186186710000038 accuracy 0.53719723183391

Epoch 6/30
----------
